In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [2]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수 67: PER 71:자본금 72: 액면가
        rqField = [0, 4, 20, 67, 71, 72]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
            per = self.objRq.GetDataValue(3, i)  # per
            base = self.objRq.GetDataValue(4, i)  # base
            face = self.objRq.GetDataValue(5, i)  # 액면가
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
                print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt, per, base, face)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        allcodelist = ("U001", "U179", "U180") + codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [3]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2973, 거래소 1555, 코스닥 1415
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
A005930 297593632700000
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (2973 종목)
삼성전자 상장주식수: 5,969,782, 시가총액 297,593,632,700,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 60,132,995,349,000
삼성바이오로직스 상장주식수: 66,165,000, 시가총액 38,905,020,000,000
삼성전자우 상장주식수: 822,886,700, 시가총액 35,178,406,425,000
NAVER 상장주식수: 164,263,395, 시가총액 32,442,020,512,500
셀트리온 상장주식수: 134,706,932, 시가총액 28,625,223,050,000
LG화학 상장주식수: 70,592,343, 시가총액 24,954,393,250,500
LG생활건강 상장주식수: 15

태경비케이 상장주식수: 27,583,100, 시가총액 101,643,723,500
대원 상장주식수: 11,587,630, 시가총액 101,391,762,500
싸이맥스 상장주식수: 10,924,243, 시가총액 100,940,005,320
서한 상장주식수: 100,894,865, 시가총액 100,894,865,000
정상제이엘에스 상장주식수: 15,677,552, 시가총액 100,806,659,360
인터파크홀딩스 상장주식수: 58,500,000, 시가총액 100,620,000,000
라이트론 상장주식수: 18,553,773, 시가총액 100,561,449,660
YBM넷 상장주식수: 16,312,697, 시가총액 100,486,213,520
디와이피엔에프 상장주식수: 10,713,625, 시가총액 100,172,393,750
브이원텍 상장주식수: 15,050,580, 시가총액 100,086,357,000
금호에이치티 상장주식수: 25,744,381, 시가총액 99,888,198,280
조선선재 상장주식수: 1,257,651, 시가총액 99,857,489,400
예스24 상장주식수: 17,200,000, 시가총액 99,588,000,000
SBI인베스트먼트 상장주식수: 162,066,575, 시가총액 99,508,877,050
모바일어플라이언스 상장주식수: 30,000,000, 시가총액 99,450,000,000
대정화금 상장주식수: 7,100,391, 시가총액 99,405,474,000
서호전기 상장주식수: 5,150,000, 시가총액 99,395,000,000
에이에프더블류 상장주식수: 10,128,444, 시가총액 99,360,035,640
신한 S&P500 VIX S/T 선물 ETN B 상장주식수: 3,000,000, 시가총액 99,345,000,000
미원화학 상장주식수: 2,199,268, 시가총액 99,186,986,800
액트 상장주식수: 20,839,763, 시가총액 98,988,874,250
코세스 상장주식수: 16,851,962, 시가총액 

케이비제18호스팩 상장주식수: 7,822,000, 시가총액 15,644,000,000
QV 에너지 TOP5 ETN 상장주식수: 2,000,000, 시가총액 15,590,000,000
삼성 Alerian 에너지인프라 MLP ETN 상장주식수: 3,000,000, 시가총액 15,435,000,000
부산주공 상장주식수: 42,397,604, 시가총액 15,432,727,856
엔에이치스팩12호 상장주식수: 7,510,000, 시가총액 15,245,300,000
TIGER S&P글로벌헬스케어(합성) 상장주식수: 1,000,000, 시가총액 15,200,000,000
하나금융11호스팩 상장주식수: 5,401,000, 시가총액 15,176,810,000
QV 자동차 TOP5 ETN 상장주식수: 2,000,000, 시가총액 15,160,000,000
KBSTAR 중소형고배당 상장주식수: 1,950,000, 시가총액 15,151,500,000
QV 의료 TOP5 ETN 상장주식수: 2,000,000, 시가총액 15,140,000,000
삼성스팩2호 상장주식수: 7,550,000, 시가총액 15,137,750,000
TIGER 단기선진하이일드(합성 H) 상장주식수: 1,400,000, 시가총액 15,113,000,000
삼성 미국 중소형 가치주 ETN 상장주식수: 2,000,000, 시가총액 15,080,000,000
TIGER 코스피고배당 상장주식수: 1,590,000, 시가총액 15,065,250,000
KODEX 미국달러선물인버스2X 상장주식수: 1,770,000, 시가총액 15,045,000,000
KINDEX S&P아시아TOP50 상장주식수: 1,350,000, 시가총액 14,991,750,000
하이트론 상장주식수: 5,530,735, 시가총액 14,988,291,850
KODEX 200 중소형 상장주식수: 2,000,000, 시가총액 14,950,000,000
쌍용양회우 상장주식수: 1,543,685, 시가총액 14,942,870,800
TRUE 섹터탑픽 ETN

In [4]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [5]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    per = item[1][2]
    eps = item[1][3]
    face = item[1][4]
    
    MarketTotal.append([name, item[0], listed, markettot, per, eps, face])

In [6]:
import pandas as pd

In [7]:
format(1e+10, ",")

'10,000,000,000.0'

In [8]:
format(1e+12, ",")

'1,000,000,000,000.0'

In [9]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [10]:
len(pdMarketTotal)

2973

In [11]:
pdMarketTotal.to_csv("../data/KRX_LIST.csv", index=None)

In [12]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization,per,base,face".split(",")

In [13]:
pdMarketTotal.loc[pdMarketTotal.Code == "U001"]

,Name,Code,Stock,capitalization,per,base,face
2970,종합주가지수,U001,0,0.0,0.0,0,0


In [14]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 200선물인버스2X"]

,Name,Code,Stock,capitalization,per,base,face
99,KODEX 200선물인버스2X,A252670,304300000,2.054025e+12,0.0,0,0


In [15]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization,per,base,face
1443,KODEX 미국S&P500선물(H),A219480,5800000,7.899600e+10,0.0,0,0


In [16]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization,per,base,face
1443,KODEX 미국S&P500선물(H),A219480,5800000,7.899600e+10,0.0,0,0


In [17]:
pdMarketTotal.loc[pdMarketTotal.Code == "A950140"]

,Name,Code,Stock,capitalization,per,base,face
1383,잉글우드랩,A950140,19867866,8.384239e+10,37.009998,0,0


In [18]:
pdMarketTotal.loc[pdMarketTotal.Code == "A900140"]

,Name,Code,Stock,capitalization,per,base,face
587,엘브이엠씨홀딩스,A900140,51120439,2.515126e+11,0.0,12,0


In [19]:
pdMarketTotal.loc[pdMarketTotal.Code == "A005930"]

,Name,Code,Stock,capitalization,per,base,face
0,삼성전자,A005930,5969782,2.975936e+14,15.74,778046,100


In [20]:
# pdMarketTotal_not_etf = pdMarketTotal.loc[(
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KODEX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("TIGER") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KBSTAR") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("HANARO") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ARIRANG") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KINDEX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KOSEF") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("SMART") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("TREX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("OTM") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ATM") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ETN") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ETF") < 0)) 
# )]

In [21]:
pdMarketTotal = pdMarketTotal[pdMarketTotal.Code.apply(lambda x: not x.endswith("K"))]
pdMarketTotal = pdMarketTotal[pdMarketTotal.Code.apply(lambda x: not x.startswith("Q"))]

In [22]:
out = pdMarketTotal.loc[(
    (pdMarketTotal.Stock == 0) |
    ((pdMarketTotal.face == 0) & (pdMarketTotal.base == 0) & (pdMarketTotal.per == 0) & (pdMarketTotal.Stock > 1e+5) & (pdMarketTotal.capitalization > 1e+10)) |
    ((pdMarketTotal.Stock > 1e+7) & (pdMarketTotal.capitalization > 1e+12))
), "Name,Code,Stock,capitalization".split(",")]

In [23]:
out

,Name,Code,Stock,capitalization
1,SK하이닉스,A000660,728002365,6.013300e+13
2,삼성바이오로직스,A207940,66165000,3.890502e+13
3,삼성전자우,A005935,822886700,3.517841e+13
4,NAVER,A035420,164263395,3.244202e+13
5,셀트리온,A068270,134706932,2.862522e+13
...,...,...,...,...
2616,KODEX TRF5050,A329660,1000000,1.011000e+10
2617,KODEX TRF7030,A329670,1000000,1.009000e+10
2970,종합주가지수,U001,0,0.000000e+00
2971,KOSPI200선물지수,U179,0,0.000000e+00


In [24]:
out.loc[out.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization
1443,KODEX 미국S&P500선물(H),A219480,5800000,7.899600e+10


In [25]:
out.to_csv("../data/SELECT_LIST.csv", index=None)